In [1]:
import nltk

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\samgh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
import pandas as pd
df=pd.read_csv('7500covid.csv')


In [18]:
from nltk.corpus import stopwords, words
from nltk.tokenize import word_tokenize
import nltk
import re
import string
import pandas as pd
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt_tab')
# Initialize stop words, English words corpus, and stemmer
stop_words = set(stopwords.words('english'))
english_words = set(words.words())
stop_words = set(stopwords.words("english"))

def preprocess(text):
    text = text.lower()

    text = re.sub(r"@\w+", "", text)

    text = re.sub(r"#(\w+)", r"\1", text)

    text = re.sub(r"http\S+|www\S+", "", text)

    text = text.translate(str.maketrans("", "", string.punctuation))

    text = ''.join([i for i in text if not i.isdigit()])

    tokens = word_tokenize(text)

    tokens = [word for word in tokens if word not in stop_words]

    return ' '.join(tokens)

df['Cleantext'] = df['original_text'].apply(preprocess)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\samgh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samgh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\samgh\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\samgh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer


label_mapping_Y = {'neg': 'negative', 'neu': 'neutral', 'pos': 'positive'}
Y_std = df['sentiment'].replace(label_mapping_Y)
X = df['Cleantext']


X_train, X_test, y_train, y_test = train_test_split(
    X, Y_std, test_size=1500, random_state=42, stratify=Y_std
)

print(f'(X_train: {len(X_train)}\nX_test: {len(X_test)})')


label_encoder = LabelEncoder()
label_encoder.fit(Y_std)  

y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

print("Label mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


analyzer = SentimentIntensityAnalyzer()

def vader_to_class(text):
    compound = analyzer.polarity_scores(text)['compound']
    if compound >= 0.05:
        return 'positive'
    elif compound <= -0.05:
        return 'negative'
    else:
        return 'neutral'


y_train_pred = X_train.apply(vader_to_class)
y_test_pred = X_test.apply(vader_to_class)


y_train_pred_num = label_encoder.transform(y_train_pred)
y_test_pred_num = label_encoder.transform(y_test_pred)


y_full_true = np.concatenate([y_train_encoded, y_test_encoded])
y_full_pred_num = np.concatenate([y_train_pred_num, y_test_pred_num])


def evaluate_metrics(y_true, y_pred):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'f1_score': f1_score(y_true, y_pred, average='weighted', zero_division=0)
    }


train_metrics = evaluate_metrics(y_train_encoded, y_train_pred_num)
test_metrics = evaluate_metrics(y_test_encoded, y_test_pred_num)
full_metrics = evaluate_metrics(y_full_true, y_full_pred_num)


print("Train metrics:", train_metrics)
print("Test metrics:", test_metrics)
print("Full dataset metrics:", full_metrics)


comparison = pd.DataFrame({
    'text': X_test.iloc[:10],
    'true_label': y_test.iloc[:10],
    'vader_pred': y_test_pred.iloc[:10]
})
print(comparison)


(X_train: 6000
X_test: 1500)
Label mapping: {'negative': np.int64(0), 'neutral': np.int64(1), 'positive': np.int64(2)}
Train metrics: {'accuracy': 0.73, 'precision': 0.7319217657377514, 'recall': 0.73, 'f1_score': 0.728249353070612}
Test metrics: {'accuracy': 0.734, 'precision': 0.7364382178467752, 'recall': 0.734, 'f1_score': 0.7321871933276178}
Full dataset metrics: {'accuracy': 0.7308, 'precision': 0.7327493396785993, 'recall': 0.7308, 'f1_score': 0.7290203014587685}
                                                   text true_label vader_pred
1152  yet voted tax scam billionaires corporations a...   negative   negative
837   rt seven point action plan combat present cris...   negative   negative
4932  rt ’ despicable radical proabortion left amp p...    neutral   negative
488   rt day stand india stand humanity feed many pe...   negative   negative
6517  amidst covid chaos would like thank lord bless...   positive   positive
3031                 rt teaming whored flagkeep reading  

In [23]:
from sklearn.metrics import classification_report

# Class names
class_names = label_encoder.classes_

# Train
print("Train classification report:")
print(classification_report(y_train_encoded, y_train_pred_num, target_names=class_names, zero_division=0))

# Test
print("Test classification report:")
print(classification_report(y_test_encoded, y_test_pred_num, target_names=class_names, zero_division=0))

# Full dataset
print("Full dataset classification report:")
print(classification_report(y_full_true, y_full_pred_num, target_names=class_names, zero_division=0))


Train classification report:
              precision    recall  f1-score   support

    negative       0.76      0.75      0.75      2000
     neutral       0.74      0.63      0.68      2000
    positive       0.70      0.81      0.75      2000

    accuracy                           0.73      6000
   macro avg       0.73      0.73      0.73      6000
weighted avg       0.73      0.73      0.73      6000

Test classification report:
              precision    recall  f1-score   support

    negative       0.74      0.76      0.75       500
     neutral       0.76      0.63      0.69       500
    positive       0.71      0.81      0.76       500

    accuracy                           0.73      1500
   macro avg       0.74      0.73      0.73      1500
weighted avg       0.74      0.73      0.73      1500

Full dataset classification report:
              precision    recall  f1-score   support

    negative       0.75      0.75      0.75      2500
     neutral       0.74      0.63   